In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
lrdf = pd.read_csv("Tai_Logistic_Regression/Churn Risk.csv")
lrdf.head()

,CustomerID,prob_churn
0,2789-HQBOU,0.864418
1,7398-SKNQZ,0.864418
2,7363-QTBIW,0.864418
3,5216-WASFJ,0.864418
4,0480-BIXDE,0.864418


In [3]:
rfdf = pd.read_csv("Random Forest/RF_churn_risk.csv")
rfdf.head()

,CustomerID,Churn Probability
0,2137-DQMEV,0.091893
1,4636-TVXVG,0.026269
2,2480-JZOSN,0.556399
3,9050-IKDZA,0.815906
4,9397-TZSHA,0.054608


In [4]:
df = rfdf.merge(lrdf, on="CustomerID")
df.head()

,CustomerID,Churn Probability,prob_churn
0,2137-DQMEV,0.091893,0.057343
1,4636-TVXVG,0.026269,0.038912
2,2480-JZOSN,0.556399,0.396576
3,9050-IKDZA,0.815906,0.771874
4,9397-TZSHA,0.054608,0.037140


In [5]:
df.rename(columns={"Churn Probability":"RF Churn Risk", "prob_churn":"Tai LR Churn Risk"}, inplace=True)

In [6]:
sylrdf = pd.read_csv("SumYe_Logistic_Regression/churn_risk.csv")
sylrdf.head()
sylrdf.rename(columns={"Churn Probability":"SumYe LR Churn Risk"}, inplace=True)

In [7]:
df = df.merge(sylrdf, on="CustomerID")

In [8]:
df["min"] = df.loc[:,"RF Churn Risk":"SumYe LR Churn Risk"].min(axis=1)

In [9]:
df["max"] = df.loc[:,"RF Churn Risk":"SumYe LR Churn Risk"].max(axis=1)

In [10]:
df["gap"] = df["max"] - df["min"]

In [11]:
df.sort_values(by="gap", ascending=False)

,CustomerID,RF Churn Risk,Tai LR Churn Risk,SumYe LR Churn Risk,min,max,gap
2723,8756-RDDLT,0.346735,0.662176,0.196174,0.196174,0.662176,0.466002
860,3893-JRNFS,0.374171,0.706500,0.249282,0.249282,0.706500,0.457219
2293,9577-WJVCQ,0.262238,0.535400,0.122722,0.122722,0.535400,0.412678
2984,7872-RDDLZ,0.351086,0.654877,0.243492,0.243492,0.654877,0.411385
5083,8548-AWOFC,0.305624,0.695821,0.292624,0.292624,0.695821,0.403197
...,...,...,...,...,...,...,...
1289,2931-XIQBR,0.154095,0.153627,0.149232,0.149232,0.154095,0.004863
1791,1264-FUHCX,0.246397,0.244355,0.242193,0.242193,0.246397,0.004204
4413,4137-BTIKL,0.736585,0.736623,0.733111,0.733111,0.736623,0.003513
209,1776-SPBWV,0.042019,0.038912,0.042307,0.038912,0.042307,0.003395


In [12]:
df.gap.mean()

0.10097740122916136

In [13]:
df["combined"] = (0.7771 * df["RF Churn Risk"] + 0.8214 * df["SumYe LR Churn Risk"] + 0.8112 * df["Tai LR Churn Risk"]) / 2.4097

In [14]:
df = df.sort_values("combined", ascending=False)

In [15]:
combined_top_500 = list(df.iloc[:500]["CustomerID"])
sumye_top_500 = []

In [16]:
with open("SumYe_Logistic_Regression/most_likely_churners.txt", "r") as file:
    for id in file:
        id = id.strip()
        sumye_top_500.append(id)

In [17]:
new_ids = [id for id in combined_top_500 if id not in sumye_top_500]

In [20]:
with open("combined_top_500.txt", "w") as file:
    for id in combined_top_500:
        file.write(f"{id}\n")